<!--BOOK_INFORMATION-->
<a href="https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv" target="_blank"><img align="left" src="data/cover.jpg" style="width: 76px; height: 100px; background: white; padding: 1px; border: 1px solid black; margin-right:10px;"></a>
*This notebook contains an excerpt from the upcoming book [Machine Learning for OpenCV](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv) by Michael Beyeler (expected Aug 2017).
The code is released under the [MIT license](https://opensource.org/licenses/MIT),
and is available on [GitHub](https://github.com/mbeyeler/opencv-machine-learning).*

*Note that this excerpt contains only the raw code - the book is rich with additional explanations and illustrations.
If you find this content useful, please consider supporting the work by
[buying the book](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-opencv)!*

<!--NAVIGATION-->
< [None](10.00-Combining-Different-Algorithms-Into-an-Ensemble.ipynb) | [Contents](../README.md) | [None](12.00-Conclusion.ipynb) >

# Selecting the Right Model with Hyper-Parameter Tuning

## Model validation the wrong way

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()

In [2]:
import numpy as np
X = iris.data.astype(np.float32)
y = iris.target

In [3]:
import cv2
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)

In [4]:
knn.train(X, cv2.ml.ROW_SAMPLE, y)
_, y_hat = knn.predict(X)

In [5]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_hat)

1.0

## Model validation the right way

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=37,
                                                    train_size=0.5)

In [8]:
knn = cv2.ml.KNearest_create()
knn.setDefaultK(1)
knn.train(X_train, cv2.ml.ROW_SAMPLE, y_train)

True

In [9]:
_, y_test_hat = knn.predict(X_test)
accuracy_score(y_test, y_test_hat)

0.88

## Cross-validation

In [10]:
knn.train(X_train, cv2.ml.ROW_SAMPLE, y_train)
_, y_fold1 = knn.predict(X_test)
accuracy_score(y_test, y_fold1)

0.88

In [11]:
knn.train(X_test, cv2.ml.ROW_SAMPLE, y_test)
_, y_fold2 = knn.predict(X_train)
accuracy_score(y_train, y_fold2)

0.92000000000000004

In [12]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([ 0.96666667,  0.96666667,  0.93333333,  0.93333333,  1.        ])

In [14]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut(len(X)))
scores

C:\Users\mbeyeler\Miniconda3\envs\OpenCVML\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [15]:
scores.mean()

0.95999999999999996

## Selecting the Best Model

## Trying to wrap the OpenCV classifier so scikit-learn can understand it

In [16]:
X.shape

(150, 4)

In [17]:
class KnnWrapper(object):
    def __init__(self, k):
        self.knn = cv2.ml.KNearest_create()
        self.k = k
        self.knn.setDefaultK(k)
    
    def fit(self, X, y):
        self.knn.train(X, cv2.ml.ROW_SAMPLE, y)
        return self.knn
    
    def predict(self, X):
        print(X.shape)
        _, y_hat = self.knn.predict(X)
        return y_hat.astype(np.float32)
    
    def score(self, y, y_hat):
        print(y.shape)
        print(y_hat.shape)
        print(y.dtype)
        return accuracy_score(y[:, 0].astype(np.float32), y_hat.astype(np.float32))
    
    def get_params(self, deep=True):
        return {'k': self.k}
        

In [18]:
myknn = KnnWrapper(1)

In [19]:
from sklearn.model_selection import cross_val_score
cross_val_score(myknn, X, y, cv=5)

(30, 4)
(30,)
float32


ValueError: Can't handle mix of continuous and binary

<!--NAVIGATION-->
< [None](10.00-Combining-Different-Algorithms-Into-an-Ensemble.ipynb) | [Contents](../README.md) | [None](12.00-Conclusion.ipynb) >